<a href="https://colab.research.google.com/github/jonlwowski012/StockSellRulesChecker/blob/master/StockScreener/Screener.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! git clone https://github.com/jonlwowski012/StockSellRulesChecker.git
! pip install -r StockSellRulesChecker/StockScreener/requirements.txt
! pip install -U git+https://github.com/mariostoev/finviz

fatal: destination path 'StockSellRulesChecker' already exists and is not an empty directory.
  Cloning https://github.com/mariostoev/finviz to /tmp/pip-req-build-y_h253u2
  Running command git clone -q https://github.com/mariostoev/finviz /tmp/pip-req-build-y_h253u2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for finviz: filename=finviz-1.3.4-cp37-none-any.whl size=16979 sha256=e7c9b60a7e25c98db9bd4264fdb271fdaf415adef1bef3611de4b55a097d4a31
  Stored in directory: /tmp/pip-ephem-wheel-cache-wlhx8qvu/wheels/98/cc/6a/68893eb662f0f196f8e153e9ce4424a8ae9e1f022fd64e4d6d
Successfully built finviz
  Found existing installation: finviz 1.3.4
    Uninstalling finviz-1.3.4:
      Successfully uninstalled finviz-1.3.4


In [2]:
from finviz.screener import Screener
import nest_asyncio
import finviz
import pandas_datareader as pdr
import yfinance as yf
yf.pdr_override() # <== that's all it takes :-)

import datetime
import stockquotes
import pandas as pd
import tqdm
nest_asyncio.apply()

import time

In [3]:
eps_5year_over5pct = True
eps_qoq_over5pct = True
eps_yoy_over5pct = True
sales_5years_over5pct = True
sales_qoq_over5pct = True

filters = []
if eps_5year_over5pct:
  filters.append('fa_eps5years_o5')
if eps_qoq_over5pct:
  filters.append('fa_epsqoq_o5')
if eps_yoy_over5pct:
  filters.append('fa_epsyoy_o5')
if sales_5years_over5pct:
  filters.append('fa_sales5years_o5')
if sales_qoq_over5pct:
  filters.append('fa_salesqoq_o5')

stock_list = Screener(filters=filters, table='Performance', order='price')  

In [4]:
print(len(stock_list.data))

367


In [5]:
def moving_average(yahoo_df, days, delta=0):
    df = yahoo_df.reset_index()
    curr_date = df['Date'].max() - datetime.timedelta(days=delta)
    end_date = curr_date - datetime.timedelta(days=days) - datetime.timedelta(days=delta)
    
    df_days = df[df['Date'] >= end_date]
    df_days = df_days[df_days['Date'] <= curr_date]
    return float(df_days['Close'].mean())

def week52_low_high(yahoo_df):
    df = yahoo_df.reset_index()
    curr_date = df['Date'].max()
    end_date = curr_date - datetime.timedelta(days=365)
    
    df_days = df[df['Date'] >= end_date]
    return float(df_days['Close'].max()), float(df_days['Close'].min())

def SMA200_slope_positive_rule(yahoo_df, days=21):
  for day in range(days):
    if moving_average(yahoo_df, days=200, delta=day) > moving_average(yahoo_df, days=200, delta=day+1):
      continue
    else:
      return False
  return True

screened_stocks = {}
for stock in tqdm.tqdm(stock_list, total=len(stock_list.data)):
  print(stock['Ticker'])
  if stock["Ticker"] == "":
    continue
  screened_stocks[stock['Ticker']] = {}
  finviz_stats = finviz.get_stock(stock['Ticker'])
  try:
      yahoo_df = pdr.get_data_yahoo(stock['Ticker'], interval = "d")
  except:
    continue

  SMA200_value = moving_average(yahoo_df, days=200)
  SMA150_value = moving_average(yahoo_df, days=150)
  SMA50_value = moving_average(yahoo_df, days=50)
  SMA200_percent = float(finviz_stats['SMA200'].replace("%",""))
  SMA50_percent = float(finviz_stats['SMA50'].replace("%",""))
  volume = float(finviz_stats['Volume'].replace(",",""))

  try:
    EPS_ttm = float(finviz_stats['EPS (ttm)'])
  except:
    EPS_ttm = 0

  try:
    EPS_QoQ_percent = float(finviz_stats['EPS Q/Q'].replace("%",""))
  except:
    EPS_QoQ_percent = 0

  try:
    Sales_QoQ_percent = float(finviz_stats['Sales Q/Q'].replace("%",""))
  except:
    Sales_QoQ_percent = 0

  try:
    inst_own = float(finviz_stats['Inst Own'].replace("%",""))
  except:
    inst_own = 0
  
  shares_outstanding = finviz_stats['Shs Outstand']
  if 'M' in shares_outstanding:
    shares_outstanding = float(shares_outstanding.replace("M", ""))*1e6
  elif 'B' in shares_outstanding:
    shares_outstanding = float(shares_outstanding.replace("B", ""))*1e9
  elif 'T' in shares_outstanding:
    shares_outstanding = float(shares_outstanding.replace("T", ""))*1e12
  else:
    shares_outstanding = 0

  prev_close = float(finviz_stats['Prev Close'].replace("$",""))
  week52_high, week52_low = week52_low_high(yahoo_df)
  
  screened_stocks[stock['Ticker']]['SMA200_value'] = SMA200_value
  screened_stocks[stock['Ticker']]['SMA150_value'] = SMA150_value
  screened_stocks[stock['Ticker']]['SMA50_value'] = SMA50_value
  screened_stocks[stock['Ticker']]['SMA200_percent'] = SMA200_percent
  screened_stocks[stock['Ticker']]['SMA50_percent'] = SMA50_percent
  screened_stocks[stock['Ticker']]['EPS_QoQ_percent'] = EPS_QoQ_percent
  screened_stocks[stock['Ticker']]['EPS_ttm'] = EPS_ttm
  screened_stocks[stock['Ticker']]['Sales_QoQ_percent'] = Sales_QoQ_percent
  screened_stocks[stock['Ticker']]['prev_close'] = prev_close
  screened_stocks[stock['Ticker']]['week52_high'] = week52_high
  screened_stocks[stock['Ticker']]['week52_low'] = week52_low
  screened_stocks[stock['Ticker']]['Inst. Ownership'] = inst_own
  screened_stocks[stock['Ticker']]['Shares Outstanding'] = shares_outstanding
  screened_stocks[stock['Ticker']]['volume'] = volume

  # volume*price > $15mm rule
  if volume*prev_close > 15000000:
    vol_price_rule = True
  else:
    vol_price_rule = False
  screened_stocks[stock['Ticker']]['vol_price_rule'] = vol_price_rule

  # EPS TTM > $1 Rule
  if EPS_ttm >= 1.0:
    eps_ttm_rule = True
  else:
    eps_ttm_rule = False
  screened_stocks[stock['Ticker']]['eps_ttm_rule'] = eps_ttm_rule

  # EPS QoQ Yearly > 18% rule
  if EPS_QoQ_percent >= 18:
    eps_QoQ_yearly_rule = True
  else:
    eps_QoQ_yearly_rule = False
  screened_stocks[stock['Ticker']]['eps_QoQ_yearly_rule'] = eps_QoQ_yearly_rule

  # Sales QoQ Yearly > 25% rule
  if Sales_QoQ_percent >= 25:
    sales_QoQ_yearly_rule = True
  else:
    sales_QoQ_yearly_rule = False
  screened_stocks[stock['Ticker']]['sales_QoQ_yearly_rule'] = sales_QoQ_yearly_rule

  # Shares Outstanding <= 25 mil
  if shares_outstanding <= 25e6:
    shares_outstanding_rule = True
  else:
    shares_outstanding_rule = False
  screened_stocks[stock['Ticker']]['shares_outstanding_rule'] = shares_outstanding_rule

  # Institutional Ownership < 35%
  if 0.05 <= inst_own <= 35:
    inst_ownership_rule = True
  else:
    inst_ownership_rule = False
  screened_stocks[stock['Ticker']]['inst_ownership_rule'] = inst_ownership_rule

  # Positive 200d MA positive
  SMA200_slope_rule = SMA200_slope_positive_rule(yahoo_df, days=21)
  screened_stocks[stock['Ticker']]['SMA200_slope_rule'] = SMA200_slope_rule
      
  # 150d MA greater than 200d MA
  if SMA150_value > SMA200_value:
      SMA150_greater_SMA200_rule = True
  else:
      SMA150_greater_SMA200_rule = False
  screened_stocks[stock['Ticker']]['SMA150_greater_SMA200_rule'] = SMA150_greater_SMA200_rule
      
  # 50d MA greater than 150d MA
  if SMA50_value > SMA150_value:
      SMA50_greater_SMA150_rule = True
  else:
      SMA50_greater_SMA150_rule = False
  screened_stocks[stock['Ticker']]['SMA50_greater_SMA150_rule'] = SMA50_greater_SMA150_rule
      
  # Close above 50d MA
  if prev_close > SMA50_value:
      close_greater_SMA50_rule = True
  else:
      close_greater_SMA50_rule = False
  screened_stocks[stock['Ticker']]['close_greater_SMA50_rule'] = close_greater_SMA50_rule
      
  # 52 week high low span rule
  if 0.75*week52_high > 1.25*week52_low:
      week52_span_rule = True
  else:
      week52_span_rule = False
  screened_stocks[stock['Ticker']]['week52_span_rule'] = week52_span_rule
  
  # Close above 52 week high - 25%
  if prev_close > 0.75*week52_high:
      close_above_52weekhigh_rule = True
  else:
      close_above_52weekhigh_rule = False
  screened_stocks[stock['Ticker']]['close_above_52weekhigh_rule'] = close_above_52weekhigh_rule

output_list = []
for stock in screened_stocks.keys():
    cols = ["Ticker"] + list(screened_stocks[stock].keys())
    temp_list = []
    temp_list.append(stock)
    for rule in screened_stocks[stock].keys():
        temp_list.append(screened_stocks[stock][rule])
    output_list.append(temp_list)
        
df_out = pd.DataFrame(output_list,columns=cols)

    


  0%|          | 0/367 [00:00<?, ?it/s]

ISR


  0%|          | 1/367 [00:03<20:55,  3.43s/it]

SMA200_slope_rule:  True
COCP


  1%|          | 2/367 [00:06<19:42,  3.24s/it]

...

SMA200_slope_rule:  False
GOOGL


100%|█████████▉| 366/367 [20:41<00:03,  3.46s/it]

SMA200_slope_rule:  True
AMZN


100%|██████████| 367/367 [20:45<00:00,  3.39s/it]

SMA200_slope_rule:  False


In [6]:
rules = [col for col in df_out.columns if "rule" in col]
df_out['Passed Tests'] = (df_out[list(rules)]).sum(1)

In [7]:
df_out.sort_values('Passed Tests', ascending=False)

,Ticker,SMA200_value,SMA150_value,SMA50_value,SMA200_percent,SMA50_percent,EPS_QoQ_percent,EPS_ttm,Sales_QoQ_percent,prev_close,week52_high,week52_low,Inst. Ownership,Shares Outstanding,volume,vol_price_rule,eps_ttm_rule,eps_QoQ_yearly_rule,sales_QoQ_yearly_rule,shares_outstanding_rule,inst_ownership_rule,SMA200_slope_rule,SMA150_greater_SMA200_rule,SMA50_greater_SMA150_rule,close_greater_SMA50_rule,week52_span_rule,close_above_52weekhigh_rule,Passed Tests
113,MPB,22.262014,23.441250,25.085555,31.12,13.90,107.9,3.11,33.5,27.31,29.410000,15.900000,21.9,8420000.0,47993.0,False,True,True,True,True,True,True,True,True,True,True,True,11
319,ESGR,197.908704,208.792692,229.037500,38.53,16.77,306.6,77.78,128.4,266.43,266.429993,123.599998,84.9,21510000.0,165818.0,True,True,True,True,True,False,True,True,True,True,True,True,11
337,UI,249.652446,276.253846,332.160833,50.73,10.59,92.5,8.05,55.5,337.74,357.070007,127.389999,7.5,62820000.0,177878.0,True,True,True,True,False,True,True,True,True,True,True,True,11
196,HZO,36.307914,39.576827,48.849445,85.85,31.35,150.7,3.98,35.3,59.83,61.770000,7.870000,0.0,22030000.0,667859.0,True,True,True,True,True,False,True,True,True,True,True,True,11
83,CBNK,13.030216,14.029615,16.689167,63.52,25.76,96.7,1.87,33.5,19.09,20.750000,9.060000,25.9,13690000.0,164609.0,False,True,True,True,True,True,True,True,True,True,True,True,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27,CXDO,6.770719,6.720048,6.470278,-4.76,-1.11,3040.8,0.42,16.2,6.44,11.290000,3.300000,8.8,17900000.0,9588.0,False,False,True,False,True,True,False,False,False,False,True,False,4
12,QTNT,5.439496,5.516250,4.762500,-31.45,-19.41,20.9,-1.10,11.4,4.04,9.260000,3.000000,92.3,101020000.0,1953003.0,False,False,True,False,False,False,False,True,False,False,True,False,3
8,IAG,3.604460,3.469904,3.275556,-13.91,-2.00,117.7,0.07,18.4,3.23,5.200000,2.000000,76.8,474700000.0,9010808.0,True,False,True,False,False,False,False,False,False,False,True,False,3
347,REGN,525.456980,507.458462,477.504722,-14.62,-3.25,47.9,30.68,11.7,474.68,658.210022,429.779999,87.8,105500000.0,1077712.0,True,True,True,False,False,False,False,False,False,False,False,False,3


In [8]:
df_out[df_out["Passed Tests"] >= 11]

,Ticker,SMA200_value,SMA150_value,SMA50_value,SMA200_percent,SMA50_percent,EPS_QoQ_percent,EPS_ttm,Sales_QoQ_percent,prev_close,week52_high,week52_low,Inst. Ownership,Shares Outstanding,volume,vol_price_rule,eps_ttm_rule,eps_QoQ_yearly_rule,sales_QoQ_yearly_rule,shares_outstanding_rule,inst_ownership_rule,SMA200_slope_rule,SMA150_greater_SMA200_rule,SMA50_greater_SMA150_rule,close_greater_SMA50_rule,week52_span_rule,close_above_52weekhigh_rule,Passed Tests
82,CFBK,15.496763,16.744231,18.943333,43.41,9.38,435.7,4.25,245.5,19.19,20.250000,9.470000,32.3,6270000.0,5578.0,False,True,True,True,True,True,True,True,True,True,True,True,11
83,CBNK,13.030216,14.029615,16.689167,63.52,25.76,96.7,1.87,33.5,19.09,20.750000,9.060000,25.9,13690000.0,164609.0,False,True,True,True,True,True,True,True,True,True,True,True,11
92,ESCA,20.580791,21.178077,21.841667,18.43,2.25,98.5,1.82,59.1,22.23,24.049999,4.750000,30.7,14030000.0,123385.0,False,True,True,True,True,True,True,True,True,True,True,True,11
113,MPB,22.262014,23.441250,25.085555,31.12,13.90,107.9,3.11,33.5,27.31,29.410000,15.900000,21.9,8420000.0,47993.0,False,True,True,True,True,True,True,True,True,True,True,True,11
117,RBNC,18.783741,20.149904,23.150556,66.04,32.56,99.0,1.89,73.0,28.17,29.330000,9.740000,32.8,16620000.0,285694.0,False,True,True,True,True,True,True,True,True,True,True,True,11
196,HZO,36.307914,39.576827,48.849445,85.85,31.35,150.7,3.98,35.3,59.83,61.770000,7.870000,0.0,22030000.0,667859.0,True,True,True,True,True,False,True,True,True,True,True,True,11
319,ESGR,197.908704,208.792692,229.037500,38.53,16.77,306.6,77.78,128.4,266.43,266.429993,123.599998,84.9,21510000.0,165818.0,True,True,True,True,True,False,True,True,True,True,True,True,11
337,UI,249.652446,276.253846,332.160833,50.73,10.59,92.5,8.05,55.5,337.74,357.070007,127.389999,7.5,62820000.0,177878.0,True,True,True,True,False,True,True,True,True,True,True,True,11
363,WTM,988.223599,1038.944904,1154.136112,23.29,5.45,686.3,224.89,197.1,1175.51,1267.520020,702.919983,88.8,3220000.0,36900.0,True,True,True,True,True,False,True,True,True,True,True,True,11
